{ “cells”: \[ { “cell_type”: “markdown”, “metadata”: {}, “source”: \[
“\# Training curves + stats (Detector + CVS)”, “”, “This notebook reads
your `history.jsonl` logs and plots:”, “- Detector: `train_loss`,
`val_map`”, “- CVS: `train_loss`, `val_loss`, `val_macro_f1`, and
per-criterion F1 (C1/C2/C3)”, “”, “It also prints the **best epoch** for
each run.” \] }, { “cell_type”: “code”, “execution_count”: null,
“metadata”: {}, “outputs”: \[\], “source”: \[ “from pathlib import
Path”, “import json”, “import pandas as pd”, “import numpy as np”,
“import matplotlib.pyplot as plt”, “”, “\# —————————-”, “\# EDIT THESE
PATHS”, “\# —————————-”, “\# Set to your coursework folder that contains
"runs/"”, “PROJECT_ROOT =
Path(r"C:/Users/TUF/COMP0220_Deep_Learning/coursework")”, “”, “\#
Baseline run folders (as in your config.toml)”, “DETECTOR_RUN =
PROJECT_ROOT / "runs" / "detector_baseline"”, “CVS_RUN = PROJECT_ROOT /
"runs" / "cvs_baseline"”, “”, “print("PROJECT_ROOT:", PROJECT_ROOT)”,
“print("DETECTOR_RUN exists:", DETECTOR_RUN.exists())”, “print("CVS_RUN
exists:", CVS_RUN.exists())” \] }, { “cell_type”: “code”,
“execution_count”: null, “metadata”: {}, “outputs”: \[\], “source”: \[
“def read_jsonl(path: Path) -\> pd.DataFrame:”, ” if not
path.exists():“,” raise FileNotFoundError(f"Missing: {path}")“,” rows =
\[\]“,” with path.open("r", encoding="utf-8") as f:“,” for line in f:“,”
line = line.strip()“,” if not line:“,” continue“,”
rows.append(json.loads(line))“,” if len(rows) == 0:“,” return
pd.DataFrame()“,” return pd.DataFrame(rows)“,”“,”“,”def
safe_float(x):“,” try:“,” return float(x)“,” except Exception:“,” return
np.nan“,”“,”“,”def add_cvs_percriterion_f1(df: pd.DataFrame) -\>
pd.DataFrame:“,” \# per_criterion is a list of 3 dicts with key ‘f1’“,”
if "per_criterion" not in df.columns:“,” return df“,” def
get_f1(idx):“,” out = \[\]“,” for v in df\["per_criterion"\].values:“,”
try:“,” out.append(float(v\[idx\]\["f1"\]))“,” except Exception:“,”
out.append(np.nan)“,” return out“,” df = df.copy()“,” df\["f1_c1"\] =
get_f1(0)“,” df\["f1_c2"\] = get_f1(1)“,” df\["f1_c3"\] = get_f1(2)“,”
return df“,”“,”“,”def best_epoch(df: pd.DataFrame, metric: str, mode:
str = "max"):“,” if df.empty or metric not in df.columns:“,” return
None“,” s = pd.to_numeric(df\[metric\], errors="coerce")“,” if
s.isna().all():“,” return None“,” i = s.idxmax() if mode == "max" else
s.idxmin()“,” row = df.loc\[i\].to_dict()“,” return row” \] }, {
“cell_type”: “markdown”, “metadata”: {}, “source”: \[ “\## Load
histories” \] }, { “cell_type”: “code”, “execution_count”: null,
“metadata”: {}, “outputs”: \[\], “source”: \[ “det_hist_path =
DETECTOR_RUN / "history.jsonl"”, “cvs_hist_path = CVS_RUN /
"history.jsonl"”, “”, “det = read_jsonl(det_hist_path)”, “cvs =
read_jsonl(cvs_hist_path)”, “cvs = add_cvs_percriterion_f1(cvs)”, “”,
“print("Detector history rows:", len(det))”, “print("CVS history rows:",
len(cvs))”, “”, “display(det.tail(5))”, “display(cvs.tail(5))” \] }, {
“cell_type”: “markdown”, “metadata”: {}, “source”: \[ “\## Best epoch
summary” \] }, { “cell_type”: “code”, “execution_count”: null,
“metadata”: {}, “outputs”: \[\], “source”: \[ “best_det =
best_epoch(det, "val_map", mode="max")”, “best_cvs = best_epoch(cvs,
"val_macro_f1", mode="max")”, “”, “print("=== Detector best (by val_map)
===")”, “print(best_det)”, “”, “print("\n=== CVS best (by val_macro_f1)
===")”, “print(best_cvs)” \] }, { “cell_type”: “markdown”, “metadata”:
{}, “source”: \[ “\## Plot Detector curves” \] }, { “cell_type”: “code”,
“execution_count”: null, “metadata”: {}, “outputs”: \[\], “source”: \[
“if det.empty:”, ” print("Detector history is empty.")“,”else:“,” det =
det.copy()“,” det\["epoch"\] = pd.to_numeric(det.get("epoch",
np.arange(1, len(det)+1)), errors="coerce")“,” det\["train_loss"\] =
pd.to_numeric(det.get("train_loss", np.nan), errors="coerce")“,”
det\["val_map"\] = pd.to_numeric(det.get("val_map", np.nan),
errors="coerce")“,”“,” plt.figure(figsize=(8, 4))“,”
plt.plot(det\["epoch"\], det\["train_loss"\], marker="o")“,”
plt.title("Detector: train loss")“,” plt.xlabel("epoch")“,”
plt.ylabel("loss")“,” plt.grid(True)“,” plt.show()“,”“,”
plt.figure(figsize=(8, 4))“,” plt.plot(det\["epoch"\], det\["val_map"\],
marker="o")“,” plt.title("Detector: val mAP@\[.5:.95\]")“,”
plt.xlabel("epoch")“,” plt.ylabel("mAP")“,” plt.grid(True)“,”
plt.show()” \] }, { “cell_type”: “markdown”, “metadata”: {}, “source”:
\[ “\## Plot CVS curves” \] }, { “cell_type”: “code”, “execution_count”:
null, “metadata”: {}, “outputs”: \[\], “source”: \[ “if cvs.empty:”, ”
print("CVS history is empty.")“,”else:“,” cvs = cvs.copy()“,”
cvs\["epoch"\] = pd.to_numeric(cvs.get("epoch", np.arange(1,
len(cvs)+1)), errors="coerce")“,” for k in \["train_loss", "val_loss",
"val_macro_f1", "f1_c1", "f1_c2", "f1_c3"\]:“,” if k in cvs.columns:“,”
cvs\[k\] = pd.to_numeric(cvs\[k\], errors="coerce")“,”“,”
plt.figure(figsize=(8, 4))“,” plt.plot(cvs\["epoch"\],
cvs\["train_loss"\], marker="o", label="train_loss")“,”
plt.plot(cvs\["epoch"\], cvs\["val_loss"\], marker="o",
label="val_loss")“,” plt.title("CVS: loss")“,” plt.xlabel("epoch")“,”
plt.ylabel("loss")“,” plt.grid(True)“,” plt.legend()“,” plt.show()“,”“,”
plt.figure(figsize=(8, 4))“,” plt.plot(cvs\["epoch"\],
cvs\["val_macro_f1"\], marker="o")“,” plt.title("CVS: val macro-F1")“,”
plt.xlabel("epoch")“,” plt.ylabel("macro-F1")“,” plt.grid(True)“,”
plt.show()“,”“,” if "f1_c1" in cvs.columns:“,” plt.figure(figsize=(8,
4))“,” plt.plot(cvs\["epoch"\], cvs\["f1_c1"\], marker="o",
label="C1")“,” plt.plot(cvs\["epoch"\], cvs\["f1_c2"\], marker="o",
label="C2")“,” plt.plot(cvs\["epoch"\], cvs\["f1_c3"\], marker="o",
label="C3")“,” plt.title("CVS: per-criterion F1")“,”
plt.xlabel("epoch")“,” plt.ylabel("F1")“,” plt.grid(True)“,”
plt.legend()“,” plt.show()” \] }, { “cell_type”: “markdown”, “metadata”:
{}, “source”: \[ “\## (Optional) Compare multiple runs automatically”,
“”, “If you create `runs/detector_*` or `runs/cvs_*`, this cell will
list them and plot `val_map` / `val_macro_f1`.” \] }, { “cell_type”:
“code”, “execution_count”: null, “metadata”: {}, “outputs”: \[\],
“source”: \[ “RUNS_ROOT = PROJECT_ROOT / "runs"”, “det_runs = sorted(\[p
for p in RUNS_ROOT.glob("detector\*") if (p /
"history.jsonl").exists()\])”, “cvs_runs = sorted(\[p for p in
RUNS_ROOT.glob("cvs\*") if (p / "history.jsonl").exists()\])”, “”,
“print("Detector runs:")”, “for p in det_runs:”, ” print(" -",
p.name)“,”“,”print("\nCVS runs:")“,”for p in cvs_runs:“,” print(" -",
p.name)“,”“,”\# Plot val_map across detector runs“,”if len(det_runs) \>
1:“,” plt.figure(figsize=(9, 4))“,” for run in det_runs:“,” df =
read_jsonl(run / "history.jsonl")“,” if "epoch" not in df.columns or
"val_map" not in df.columns:“,” continue“,” df\["epoch"\] =
pd.to_numeric(df\["epoch"\], errors="coerce")“,” df\["val_map"\] =
pd.to_numeric(df\["val_map"\], errors="coerce")“,”
plt.plot(df\["epoch"\], df\["val_map"\], marker="o", label=run.name)“,”
plt.title("Detector runs: val mAP")“,” plt.xlabel("epoch")“,”
plt.ylabel("mAP")“,” plt.grid(True)“,” plt.legend()“,”
plt.show()“,”“,”\# Plot val_macro_f1 across cvs runs“,”if len(cvs_runs)
\> 1:“,” plt.figure(figsize=(9, 4))“,” for run in cvs_runs:“,” df =
read_jsonl(run / "history.jsonl")“,” if "epoch" not in df.columns or
"val_macro_f1" not in df.columns:“,” continue“,” df\["epoch"\] =
pd.to_numeric(df\["epoch"\], errors="coerce")“,” df\["val_macro_f1"\] =
pd.to_numeric(df\["val_macro_f1"\], errors="coerce")“,”
plt.plot(df\["epoch"\], df\["val_macro_f1"\], marker="o",
label=run.name)“,” plt.title("CVS runs: val macro-F1")“,”
plt.xlabel("epoch")“,” plt.ylabel("macro-F1")“,” plt.grid(True)“,”
plt.legend()“,” plt.show()” \] } \], “metadata”: { “kernelspec”: {
“display_name”: “Python 3”, “language”: “python”, “name”: “python3” },
“language_info”: { “name”: “python”, “version”: “3.x” } }, “nbformat”:
4, “nbformat_minor”: 5 }